In [221]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [238]:
df = pd.read_excel('/Users/liyurun/Desktop/data1.xlsx')

In [239]:
df.dropna(axis=0,how='all',inplace=True)
df.dropna(axis=1,how='all',inplace=True)

df = df.drop(columns=['Unnamed: 15'])

df.dropna(axis=0,how='any',inplace=True)

df['ID']=df['ID'].astype('int')
df.head(10)

,ID,origin path[lat],origin path[lon],delivery time,配達予約時間,if timeout[origin],distance of i and i+1[origin],optimized path[lat],optimized path[lon],distance of i and i+1[optimized],predicted delivery time,optimized time cost[min],if timeout,lat [time limit],lon [time limit],distance[time limit],predicted delivery time.1,limited time cost[min],if timeout[time limit]
0,24089,35.114228,136.640980,1029.0,1020.0,0.0,245.0,35.114228,136.640980,245.0,1029.00,5.36,0.0,35.114228,136.640980,245.0,1029.00,2.72,0.0
1,24089,35.112731,136.639003,1042.0,1100.0,1.0,9084.0,35.112731,136.639003,681.0,1034.36,8.72,1.0,35.112731,136.639003,681.0,1031.72,9.28,1.0
2,24089,35.033508,136.615065,1107.0,1120.0,1.0,8437.0,35.107506,136.635094,8437.0,1043.08,4.48,1.0,35.107506,136.635094,8437.0,1041.00,6.80,1.0
3,24089,35.107506,136.635094,1107.0,1120.0,1.0,799.0,35.033508,136.615065,7653.0,1047.56,3.36,1.0,35.033508,136.615065,7653.0,1047.80,3.36,1.0
4,24089,35.100322,136.634881,1119.0,1200.0,1.0,1644.0,35.100322,136.634881,1644.0,1050.92,13.04,1.0,35.100322,136.634881,1644.0,1051.16,19.36,1.0
6,26511,35.349609,136.607749,940.0,1200.0,1.0,636.0,35.349609,136.607749,834.0,940.00,8.72,1.0,35.349609,136.607749,834.0,940.00,7.12,1.0
7,26511,35.343989,136.609028,948.0,1200.0,1.0,160.0,35.356514,136.604155,1810.0,948.72,11.20,1.0,35.356514,136.604155,1810.0,947.12,11.28,1.0
8,26511,35.343591,136.607335,953.0,1200.0,1.0,1467.0,35.340485,136.600786,2153.0,959.92,22.32,1.0,35.340485,136.600786,2153.0,958.40,23.44,1.0
9,26511,35.356514,136.604155,1019.0,1200.0,1.0,1810.0,35.323854,136.612898,2254.0,1022.24,21.60,1.0,35.323854,136.612898,2254.0,1021.84,18.16,1.0
10,26511,35.340485,136.600786,1032.0,1200.0,1.0,1159.0,35.343591,136.607335,160.0,1043.84,2.88,1.0,35.343591,136.607335,160.0,1040.00,15.84,1.0


In [240]:
df_grouped = df.groupby('ID')
id_list = list(df_grouped.size().index)

In [236]:
df_driver = df[df['ID']==id_list[0]]

In [253]:
DIS = []
for num in range(len(id_list)):
    df_driver = df[df['ID']==id_list[num]]
    #print(df_driver['lat [time limit]'])
    lat = df_driver.loc[:,'lat [time limit]'].tolist()
    lon = df_driver.loc[:,'lon [time limit]'].tolist()
    dis = [0 for i in range(len(lat))]
    for i in range(len(lat)):
        dis[i] = cal_dis(lat[i],lon[i],lat[(i+1)%len(lat)],lon[(i+1)%len(lat)])
        DIS.append(dis[i])

In [258]:
df.insert(df.shape[1],'distance of i and i+1[time limit]',DIS)

In [263]:
df = df.drop(columns=['distance[time limit]'])

In [269]:
df.head(100)

,ID,origin path[lat],origin path[lon],delivery time,配達予約時間,if timeout[origin],distance of i and i+1[origin],optimized path[lat],optimized path[lon],distance of i and i+1[optimized],predicted delivery time,optimized time cost[min],if timeout,lat [time limit],lon [time limit],predicted delivery time.1,limited time cost[min],if timeout[time limit],distance of i and i+1[time limit]
0,24089,35.114228,136.640980,1029.0,1020.0,0.0,245.0,35.114228,136.640980,245.0,1029.00,5.36,0.0,35.114228,136.640980,1029.00,2.72,0.0,245
1,24089,35.112731,136.639003,1042.0,1100.0,1.0,9084.0,35.112731,136.639003,681.0,1034.36,8.72,1.0,35.112731,136.639003,1031.72,9.28,1.0,681
2,24089,35.033508,136.615065,1107.0,1120.0,1.0,8437.0,35.107506,136.635094,8437.0,1043.08,4.48,1.0,35.107506,136.635094,1041.00,6.80,1.0,8437
3,24089,35.107506,136.635094,1107.0,1120.0,1.0,799.0,35.033508,136.615065,7653.0,1047.56,3.36,1.0,35.033508,136.615065,1047.80,3.36,1.0,7653
4,24089,35.100322,136.634881,1119.0,1200.0,1.0,1644.0,35.100322,136.634881,1644.0,1050.92,13.04,1.0,35.100322,136.634881,1051.16,19.36,1.0,1644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,132060,35.403758,136.634688,959.0,1000.0,1.0,2562.0,35.403758,136.634688,3035.0,958.80,2.48,1.0,35.403758,136.634688,957.52,3.20,1.0,3035
106,132060,35.412135,136.608378,1011.0,1020.0,1.0,545.0,35.423304,136.611357,2198.0,1001.28,7.52,1.0,35.423304,136.611357,1000.72,9.36,1.0,2198
107,132060,35.417002,136.607694,1018.0,1000.0,0.0,102.0,35.441797,136.602846,891.0,1008.80,39.76,1.0,35.441797,136.602846,1010.08,8.08,1.0,891
108,132060,35.417917,136.607574,1026.0,1020.0,0.0,345.0,35.433789,136.602755,911.0,1048.56,16.16,0.0,35.433789,136.602755,1018.16,8.56,1.0,911


In [215]:
def cal_dis(lat1, lon1, lat2, lon2):
    lat1 = (math.pi / 180) * lat1
    lat2 = (math.pi / 180) * lat2
    lon1 = (math.pi / 180) * lon1
    lon2 = (math.pi / 180) * lon2

    R = 6378.137
    if (lat1 != lat2) & (lon1 != lon2) :
        d = math.acos(math.sin(lat1) * math.sin(lat2) + math.cos(lat1) * math.cos(lat2) * math.cos(lon2 - lon1)) * R * 1000
    else:
        d = 0
    return int(d)

In [276]:
POSITION_NUM = np.zeros(len(id_list))

ORIGIN_TIME_COST = np.zeros(len(id_list))
ORIGIN_COUNT_POSITION = np.zeros(len(id_list))
ORIGIN_COUNT_POSITION_RATE = np.zeros(len(id_list))
ORIGIN_DIS = np.zeros(len(id_list))

OPTIMIZED_TIME_COST = np.zeros(len(id_list))
OPTIMIZED_COUNT_POSITION = np.zeros(len(id_list))
OPTIMIZED_COUNT_POSITION_RATE = np.zeros(len(id_list))
OPTIMIZED_DIS = np.zeros(len(id_list))

LIMIT_TIME_COST = np.zeros(len(id_list))
LIMIT_COUNT_POSITION = np.zeros(len(id_list))
LIMIT_COUNT_POSITION_RATE = np.zeros(len(id_list))
LIMIT_DIS = np.zeros(len(id_list))


for num in range(len(id_list)):
    df_driver = df[df['ID']==id_list[num]]
    count_position = df_driver.shape[0]

    delivery_time_min = df_driver.loc[:,"delivery time"].min()
    delivery_time_max = df_driver.loc[:,"delivery time"].max()
    origin_time_cost = timeTominute(delivery_time_max-delivery_time_min)
    sum_distances = df_driver.loc[:,'distance of i and i+1[origin]'].sum()
    origin_count_meet_position = df_driver.loc[:,'if timeout[origin]'].sum()
    origin_count_opsition_rate = origin_count_meet_position/count_position

    #optimezed data
    delivery_optimizedpath_time_max=df_driver.loc[:,'predicted delivery time'].max()
    delivery_optimizedpath_time_min=df_driver.loc[:,'predicted delivery time'].min()
    optimized_time_cost = timeTominute(delivery_optimizedpath_time_max)-timeTominute(delivery_optimizedpath_time_min)
    sum_optimized_distances = df_driver.loc[:,'distance of i and i+1[optimized]'].sum()
    optimized_count_meet_position = df_driver.loc[:,'if timeout'].sum()
    optimized_count_opsition_rate = optimized_count_meet_position/count_position

    #time limited data
    delivery_time_limit_max = df_driver.loc[:,'predicted delivery time.1'].max()
    delivery_time_limit_min = df_driver.loc[:,'predicted delivery time.1'].min()
    time_limit_cost = timeTominute(delivery_time_limit_max)-timeTominute(delivery_time_limit_min)
    sum_limit_distances = df_driver.loc[:,'distance of i and i+1[time limit]'].sum()
    limit_count_meet_position = df_driver.loc[:,'if timeout[time limit]'].sum()
    limit_count_meet_position_rate = limit_count_meet_position/count_position
    
    POSITION_NUM[num] = count_position

    ORIGIN_TIME_COST[num] = origin_time_cost
    ORIGIN_COUNT_POSITION[num] = origin_count_meet_position
    ORIGIN_COUNT_POSITION_RATE[num] = origin_count_opsition_rate
    ORIGIN_DIS[num] = sum_distances

    OPTIMIZED_TIME_COST[num] = optimized_time_cost
    OPTIMIZED_COUNT_POSITION[num] = optimized_count_meet_position
    OPTIMIZED_COUNT_POSITION_RATE[num] = optimized_count_opsition_rate
    OPTIMIZED_DIS[num] = sum_optimized_distances

    LIMIT_TIME_COST[num] = time_limit_cost
    LIMIT_COUNT_POSITION[num] = limit_count_meet_position
    LIMIT_COUNT_POSITION_RATE[num] = limit_count_meet_position_rate
    LIMIT_DIS[num] = sum_limit_distances
    


In [277]:
POSITION_NUM

array([ 5.,  7., 15., 12., 15., 15.,  3.,  2., 14.,  6., 12.,  2.,  5.,
       17.,  6., 15., 10., 27., 20.,  9., 24., 10., 13., 29.,  6.,  4.,
       18., 15., 12., 14.,  3.,  7., 13., 10.,  2., 11.,  9., 16.])

In [274]:
for num in range(len(id_list)):
    df_driver = df[df['ID']==id_list[num]]
    count_position = df_driver.shape[0]
    print(count_position)

5
7
15
12
15
15
3
2
14
6
12
2
5
17
6
15
10
27
20
9
24
10
13
29
6
4
18
15
12
14
3
7
13
10
2
11
9
16


In [278]:
df_dic = {
    'driver ID':id_list,
    'origin distances[巡回] [m]':ORIGIN_DIS,
    'origin time cost(first position to last position) [min]':ORIGIN_TIME_COST,
    'origin on time positions':ORIGIN_COUNT_POSITION,
    'origin on time positions rate %':ORIGIN_COUNT_POSITION_RATE,
    
    'optimized distances[巡回] [m]':OPTIMIZED_DIS,
    'optimized time cost(first position to last position) [min]':OPTIMIZED_TIME_COST,
    'optimized on time positions':OPTIMIZED_COUNT_POSITION,
    'optimized on time positions rate %':OPTIMIZED_COUNT_POSITION_RATE,

    'time limited distances[巡回] [m]':LIMIT_DIS,
    'time limited time cost(first position to last position) [min]':LIMIT_TIME_COST,
    'time limited on time positions':LIMIT_COUNT_POSITION,
    'time limited on time positions rate %':LIMIT_COUNT_POSITION_RATE,
    
    'position number':POSITION_NUM
}
output_df = pd.DataFrame(df_dic)
output_df

,driver ID,origin distances[巡回] [m],origin time cost(first position to last position) [min],origin on time positions,origin on time positions rate %,optimized distances[巡回] [m],optimized time cost(first position to last position) [min],optimized on time positions,optimized on time positions rate %,time limited distances[巡回] [m],time limited time cost(first position to last position) [min],time limited on time positions,time limited on time positions rate %,position number
0,24089,20209.0,90.0,4.0,0.800000,18660.0,21.92,4.0,0.800000,18660.0,22.16,4.0,0.800000,5.0
1,26511,10854.0,72.0,7.0,1.000000,7900.0,73.04,7.0,1.000000,7900.0,83.36,7.0,1.000000,7.0
2,31727,256970.0,76.0,15.0,1.000000,252525.0,118.32,13.0,0.866667,255457.0,93.84,14.0,0.933333,15.0
3,54056,15330.0,134.0,12.0,1.000000,10964.0,108.56,10.0,0.833333,14488.0,66.32,12.0,1.000000,12.0
4,55996,8795.0,80.0,15.0,1.000000,7906.0,97.84,13.0,0.866667,9043.0,103.28,13.0,0.866667,15.0
5,56267,120814.0,187.0,4.0,0.266667,116743.0,120.56,9.0,0.600000,122078.0,101.92,13.0,0.866667,15.0
6,115430,212389.0,0.0,3.0,1.000000,212389.0,19.60,3.0,1.000000,212389.0,19.60,3.0,1.000000,3.0
7,131960,36.0,0.0,2.0,1.000000,36.0,2.08,2.0,1.000000,36.0,2.08,2.0,1.000000,2.0
8,131970,4850.0,82.0,14.0,1.000000,4616.0,75.28,14.0,1.000000,4616.0,105.44,14.0,1.000000,14.0
9,131994,23488.0,203.0,4.0,0.666667,13945.0,74.96,6.0,1.000000,13945.0,38.56,6.0,1.000000,6.0


5

In [162]:
def timeTominute(time):
    return time//100*60 + time%100

In [ ]:
for idx in range(17,38):
    df_driver = df[df['ID']==id_list[idx]]

    optimized_lat = df_driver['optimized path[lat]'].tolist()
    optimized_ptime = df_driver['predicted delivery time'].tolist()
    predict_time = df_driver['配達予約時間'].tolist()

    if_optimezed_timeout = np.zeros(len(optimized_lat))
    optimized_predict_time = np.zeros(len(optimized_lat))

    for i in range(len(if_optimezed_timeout)):
        optimized_predict_time[i] = df_driver[df_driver['origin path[lat]']==optimized_lat[i]]['配達予約時間'].values[0]
        if optimized_predict_time[i] >= optimized_ptime[i]:
            if_optimezed_timeout[i] = 1
        else:
            if_optimezed_timeout[i] = 0
    print('No.{}'.format(idx))
    print('id:{}'.format(id_list[idx]))
    print('if list =: {}'.format(if_optimezed_timeout))

In [150]:
df_driver.head()

,ID,origin path[lat],origin path[lon],delivery time,配達予約時間,distance of i and i+1[origin],optimized path[lat],optimized path[lon],distance of i and i+1[optimized],predicted delivery time,optimized time cost[min],if timeout,lat [time limit],lon [time limit],distance[time limit],predicted delivery time.1,limited time cost[min]
454,861633,35.346775,136.633581,919.0,1000.0,315.0,35.346775,136.633581,315.0,919.00,4.88,1.0,35.346775,136.633581,315.0,919.00,3.12
455,861633,35.347971,136.630431,925.0,1000.0,346.0,35.347971,136.630431,287.0,923.88,7.68,1.0,35.347971,136.630431,287.0,922.12,2.80
456,861633,35.347191,136.634129,937.0,1000.0,542.0,35.350487,136.629735,82.0,931.56,4.56,1.0,35.350487,136.629735,82.0,924.92,1.84
457,861633,35.350487,136.629735,943.0,1000.0,82.0,35.350792,136.630558,389.0,936.12,3.04,1.0,35.350792,136.630558,389.0,926.76,16.16
458,861633,35.350792,136.630558,946.0,1000.0,290.0,35.353780,136.628317,122.0,939.16,2.72,1.0,35.353780,136.628317,122.0,942.92,1.84


In [272]:
df.to_csv('/Users/liyurun/Desktop/data_ver2.csv')

In [282]:
output_df.to_csv('/Users/liyurun/Desktop/data_set1.csv')

,driver ID,origin distances[巡回] [m],origin time cost(first position to last position) [min],origin on time positions,origin on time positions rate %,optimized distances[巡回] [m],optimized time cost(first position to last position) [min],optimized on time positions,optimized on time positions rate %,time limited distances[巡回] [m],time limited time cost(first position to last position) [min],time limited on time positions,time limited on time positions rate %,position number
0,24089,20209.0,90.0,4.0,0.800000,18660.0,21.92,4.0,0.800000,18660.0,22.16,4.0,0.800000,5.0
1,26511,10854.0,72.0,7.0,1.000000,7900.0,73.04,7.0,1.000000,7900.0,83.36,7.0,1.000000,7.0
2,31727,256970.0,76.0,15.0,1.000000,252525.0,118.32,13.0,0.866667,255457.0,93.84,14.0,0.933333,15.0
3,54056,15330.0,134.0,12.0,1.000000,10964.0,108.56,10.0,0.833333,14488.0,66.32,12.0,1.000000,12.0
4,55996,8795.0,80.0,15.0,1.000000,7906.0,97.84,13.0,0.866667,9043.0,103.28,13.0,0.866667,15.0
5,56267,120814.0,187.0,4.0,0.266667,116743.0,120.56,9.0,0.600000,122078.0,101.92,13.0,0.866667,15.0
6,115430,212389.0,0.0,3.0,1.000000,212389.0,19.60,3.0,1.000000,212389.0,19.60,3.0,1.000000,3.0
7,131960,36.0,0.0,2.0,1.000000,36.0,2.08,2.0,1.000000,36.0,2.08,2.0,1.000000,2.0
8,131970,4850.0,82.0,14.0,1.000000,4616.0,75.28,14.0,1.000000,4616.0,105.44,14.0,1.000000,14.0
9,131994,23488.0,203.0,4.0,0.666667,13945.0,74.96,6.0,1.000000,13945.0,38.56,6.0,1.000000,6.0
